In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split

# Preprocessing Browsing Data

In [2]:
browsing_activity = pd.read_csv('experiences_browsing_data.csv')

In [3]:
browsing_activity

,_id,action,experience_id,user,__v,createdAt
0,1,recommended,65083cd6313443081a272366,1,0,2024-03-10 14:50:26
1,1,viewed,64fc2511148fd2e0b23d5031,1,0,2024-04-21 20:37:04
2,1,shared,6507ee68313443081a27234a,1,0,2024-06-09 18:35:27
3,1,liked,64fca0063d690a3e195ee937,1,0,2024-07-05 16:20:39
4,1,shared,64fc8bc73d690a3e195ee898,1,0,2024-03-03 15:30:14
...,...,...,...,...,...,...
15995,1000,purchased,65285a2e66f321cbd9ef4ba0,1000,0,2024-03-02 12:25:20
15996,1000,purchased,64dfb10e7792cee05d3328d3,1000,0,2024-09-27 17:18:48
15997,1000,shared,64fc8bc73d690a3e195ee898,1000,0,2024-05-28 17:45:30
15998,1000,purchased,65083cd6313443081a272366,1000,0,2024-08-17 00:11:10


In [4]:
browsing_activity['createdAt'] = pd.to_datetime(browsing_activity['createdAt'])

In [5]:
#browsing_activity['action'].replace(['viewed', 'liked', 'shared', 'reviewed', 'booked', 'commented', 'recommended', 'purchased', 'attended'],[1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)

In [6]:
browsing_activity = pd.get_dummies(browsing_activity, columns=['action'], prefix='action')

In [7]:
browsing_activity.head()

,_id,experience_id,user,__v,createdAt,action_attended,action_booked,action_commented,action_liked,action_purchased,action_recommended,action_reviewed,action_shared,action_viewed
0,1,65083cd6313443081a272366,1,0,2024-03-10 14:50:26,False,False,False,False,False,True,False,False,False
1,1,64fc2511148fd2e0b23d5031,1,0,2024-04-21 20:37:04,False,False,False,False,False,False,False,False,True
2,1,6507ee68313443081a27234a,1,0,2024-06-09 18:35:27,False,False,False,False,False,False,False,True,False
3,1,64fca0063d690a3e195ee937,1,0,2024-07-05 16:20:39,False,False,False,True,False,False,False,False,False
4,1,64fc8bc73d690a3e195ee898,1,0,2024-03-03 15:30:14,False,False,False,False,False,False,False,True,False


In [8]:
action_mapping = {
    'action_1': 'viewed',
    'action_2': 'liked',
    'action_3': 'shared',
    'action_4': 'reviewed',
    'action_5': 'booked',
    'action_6': 'commented',
    'action_7': 'recommended',
    'action_8': 'purchased',
    'action_9': 'attended'
}

browsing_activity = browsing_activity.rename(columns=action_mapping)

In [9]:
browsing_activity.head()

,_id,experience_id,user,__v,createdAt,action_attended,action_booked,action_commented,action_liked,action_purchased,action_recommended,action_reviewed,action_shared,action_viewed
0,1,65083cd6313443081a272366,1,0,2024-03-10 14:50:26,False,False,False,False,False,True,False,False,False
1,1,64fc2511148fd2e0b23d5031,1,0,2024-04-21 20:37:04,False,False,False,False,False,False,False,False,True
2,1,6507ee68313443081a27234a,1,0,2024-06-09 18:35:27,False,False,False,False,False,False,False,True,False
3,1,64fca0063d690a3e195ee937,1,0,2024-07-05 16:20:39,False,False,False,True,False,False,False,False,False
4,1,64fc8bc73d690a3e195ee898,1,0,2024-03-03 15:30:14,False,False,False,False,False,False,False,True,False


In [10]:
browsing_activity = browsing_activity.replace({False: 0, True: 1})

In [11]:
browsing_activity.head()

,_id,experience_id,user,__v,createdAt,action_attended,action_booked,action_commented,action_liked,action_purchased,action_recommended,action_reviewed,action_shared,action_viewed
0,1,65083cd6313443081a272366,1,0,2024-03-10 14:50:26,0,0,0,0,0,1,0,0,0
1,1,64fc2511148fd2e0b23d5031,1,0,2024-04-21 20:37:04,0,0,0,0,0,0,0,0,1
2,1,6507ee68313443081a27234a,1,0,2024-06-09 18:35:27,0,0,0,0,0,0,0,1,0
3,1,64fca0063d690a3e195ee937,1,0,2024-07-05 16:20:39,0,0,0,1,0,0,0,0,0
4,1,64fc8bc73d690a3e195ee898,1,0,2024-03-03 15:30:14,0,0,0,0,0,0,0,1,0


In [12]:
browsing_activity.drop(['_id', '__v'], axis=1, inplace=True)

In [13]:
browsing_activity.head()

,experience_id,user,createdAt,action_attended,action_booked,action_commented,action_liked,action_purchased,action_recommended,action_reviewed,action_shared,action_viewed
0,65083cd6313443081a272366,1,2024-03-10 14:50:26,0,0,0,0,0,1,0,0,0
1,64fc2511148fd2e0b23d5031,1,2024-04-21 20:37:04,0,0,0,0,0,0,0,0,1
2,6507ee68313443081a27234a,1,2024-06-09 18:35:27,0,0,0,0,0,0,0,1,0
3,64fca0063d690a3e195ee937,1,2024-07-05 16:20:39,0,0,0,1,0,0,0,0,0
4,64fc8bc73d690a3e195ee898,1,2024-03-03 15:30:14,0,0,0,0,0,0,0,1,0


In [14]:
## shuffle the row values to prevent having sequential rows of the same user
browsing_activity = browsing_activity.sample(frac=1, random_state=42)

In [15]:
browsing_activity.head()

,experience_id,user,createdAt,action_attended,action_booked,action_commented,action_liked,action_purchased,action_recommended,action_reviewed,action_shared,action_viewed
8756,65285a2e66f321cbd9ef4ba0,548,2023-10-04 15:53:49,0,0,0,0,1,0,0,0,0
4660,64fca2693d690a3e195ee94d,292,2024-01-17 16:18:55,0,0,0,0,0,0,0,0,1
6095,64fc7f3d3d690a3e195ee882,381,2023-11-01 21:44:10,0,0,0,0,0,1,0,0,0
304,64fc4dd63d690a3e195ee6ee,20,2023-12-23 09:22:11,0,0,0,0,0,0,0,0,1
8241,64fc46763d690a3e195ee6c6,516,2024-09-07 13:49:42,0,1,0,0,0,0,0,0,0


In [16]:
browsing_activity.describe()

,user,createdAt,action_attended,action_booked,action_commented,action_liked,action_purchased,action_recommended,action_reviewed,action_shared,action_viewed
count,16000.000000,16000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.000000,16000.00000
mean,500.500000,2024-03-31 18:42:23.298812416,0.110687,0.112625,0.108688,0.112125,0.111125,0.110250,0.108938,0.114062,0.11150
min,1.000000,2023-10-01 01:10:42,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,250.750000,2023-12-31 08:13:46.750000128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,500.500000,2024-03-31 18:22:50.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,750.250000,2024-07-01 03:38:22.249999872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,1000.000000,2024-09-29 23:37:36,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
std,288.684012,NaN,0.313755,0.316144,0.311256,0.315530,0.314297,0.313211,0.311571,0.317897,0.31476


## Model Training

In [17]:
"""

Matrix where rows represent users, columns represent experiences, and the values indicate whether
the user liked that experience

"""
browsing_activity = browsing_activity.pivot_table(index='user', columns='experience_id', values='action_liked', fill_value=0)

In [18]:
browsing_activity

experience_id,64dfb10e7792cee05d3328d3,64fc2511148fd2e0b23d5031,64fc3d483d690a3e195ee6a4,64fc46763d690a3e195ee6c6,64fc4dd63d690a3e195ee6ee,64fc7a953d690a3e195ee83c,64fc7f3d3d690a3e195ee882,64fc8bc73d690a3e195ee898,64fc90f13d690a3e195ee8e0,64fc9b6b3d690a3e195ee90a,64fca0063d690a3e195ee937,64fca2693d690a3e195ee94d,64fca7ce3d690a3e195ee97a,6507ee68313443081a27234a,65083cd6313443081a272366,65285a2e66f321cbd9ef4ba0
user,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [19]:
# User-Experience Interaction Matrix
browsing_activity_matrix = browsing_activity.values

In [41]:
"""
We use K-Means clustering to group users into clusters based on their liking behavior. It finds the 
optimal number of clusters using the Silhouette score and then applies K-Means with that number of clusters.
"""
max_clusters = 10
best_score = -1
best_cluster = 0
for n_clusters in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(browsing_activity_matrix)
    score = silhouette_score(browsing_activity_matrix, cluster_labels)
    if score > best_score:
        best_score = score
        best_cluster = n_clusters
        
print(f'Optimal Cluster Values: {best_cluster}')

Optimal Cluster Values: 9


In [21]:
# Fit K-Means with the best number of clusters
kmeans = KMeans(n_clusters=best_cluster, random_state=42, n_init=10)
browsing_activity['cluster'] = kmeans.fit_predict(browsing_activity_matrix)

In [22]:
"""
Generates cross-recommendations for each user based on what users in a similar cluster liked. Starts by 
identifying the cluster of the user and then selects other users in the same cluster.

For each user in the cluster, it checks their liked experiences and recommends experiences that the target user 
hasn't liked. Recommendations are based on the behavior of similar users in the same cluster.
"""
def cross_recommendations(user_id, num_recommendations=5):
    user_cluster = browsing_activity[browsing_activity.index == user_id]['cluster'].values[0]
    cluster_users = browsing_activity[browsing_activity['cluster'] == user_cluster]
    user_liked_experiences = cluster_users.loc[user_id][cluster_users.loc[user_id] > 0].index.tolist()
    
    recommendations = []
    
    for idx, row in cluster_users.iterrows():
        if idx != user_id:
            liked_experiences = row[row > 0].index.tolist()
            for exp_id in liked_experiences:
                if exp_id not in user_liked_experiences and exp_id not in recommendations:
                    recommendations.append(exp_id)
                    if len(recommendations) == num_recommendations:
                        return recommendations

In [23]:
# Example: Get cross-recommendations for user 1
user_id = 55
recommended_experiences = cross_recommendations(user_id)
print("Recommended Experiences for User", user_id, ":", recommended_experiences)

Recommended Experiences for User 55 : ['64fc3d483d690a3e195ee6a4', '6507ee68313443081a27234a', '64fca7ce3d690a3e195ee97a', '65083cd6313443081a272366', '64fca2693d690a3e195ee94d']


In [24]:
experiences_data = pd.read_json('tajriba.experiences.json')

In [26]:
experiences_data['_id'] = experiences_data['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else x)

In [27]:
experiences_data['description'] = experiences_data['description'].str.replace('\n', ' ')

In [28]:
experiences_data.head()

,_id,name,description,host,duration,location,coverImage,images,featured,adventureLevel,...,refundPolicy,canUserCancel,listedByHost,adminApprovalStage,publishedByAdmin,hostPayout,displayPrice,__v,country,signatureSelection
0,64dfb10e7792cee05d3328d3,Wasini Day Trip,Look for dolphins as you cruise the Indian Oce...,{'$oid': '643c1fd5ef7712c1efca8f03'},3,"{'name': 'Wasini Island', 'description': 'Use ...",https://firebasestorage.googleapis.com/v0/b/ta...,[],gold,5.0,...,none,False,True,none,False,4650,5555,0,{'$oid': '650fc35bac5fe18afadc174c'},NaN
1,64fc2511148fd2e0b23d5031,Game Drive at Nairobi National Park,Experience the best of both worlds at Nairobi ...,{'$oid': '643fa4c0bee42efcb799bf72'},3,"{'coordinates': [36.8172449, -1.2832533], 'nam...",https://res.cloudinary.com/dwjoypmhg/image/upl...,[https://res.cloudinary.com/dwjoypmhg/image/up...,gold,6.5,...,none,False,True,none,True,5165,6170,0,{'$oid': '650fc35bac5fe18afadc174c'},NaN
2,64fc3d483d690a3e195ee6a4,Out of Africa Experience : Karen Blixen Museum,Step into the captivating world of Karen Blixe...,{'$oid': '643fa4c0bee42efcb799bf72'},2,"{'coordinates': [36.8172449, -1.2832533], 'nam...",https://res.cloudinary.com/dwjoypmhg/image/upl...,[https://res.cloudinary.com/dwjoypmhg/image/up...,none,4.0,...,none,False,True,none,True,5165,6170,0,{'$oid': '650fc35bac5fe18afadc174c'},NaN
3,64fc46763d690a3e195ee6c6,Lunch with Elephants : Sheldrick Wildlife Trust,Embark on a transformative journey at the Shel...,{'$oid': '643fa4c0bee42efcb799bf72'},2,"{'coordinates': [36.8172449, -1.2832533], 'nam...",https://res.cloudinary.com/dwjoypmhg/image/upl...,[https://res.cloudinary.com/dwjoypmhg/image/up...,gold,6.5,...,none,False,True,none,True,5165,6170,0,{'$oid': '650fc35bac5fe18afadc174c'},NaN
4,64fc4dd63d690a3e195ee6ee,Nairobi National Museum Tour,Explore the wonders of Kenya's rich heritage a...,{'$oid': '643fa4c0bee42efcb799bf72'},2,"{'coordinates': [36.8172449, -1.2832533], 'nam...",https://res.cloudinary.com/dwjoypmhg/image/upl...,[https://res.cloudinary.com/dwjoypmhg/image/up...,none,4.5,...,none,False,True,none,True,5165,6170,0,{'$oid': '650fc35bac5fe18afadc174c'},NaN


In [49]:
# Create a DataFrame for the recommended experiences
user_id = 223
recommended_experiences_df = experiences_data[experiences_data['_id'].isin(cross_recommendations(user_id))]

In [50]:
features = ['_id', 'coverImage', 'name', 'description', 'displayPrice']
recommended_experiences_df = recommended_experiences_df[features]

In [51]:
recommended_experiences_df

,_id,coverImage,name,description,displayPrice
0,64dfb10e7792cee05d3328d3,https://firebasestorage.googleapis.com/v0/b/ta...,Wasini Day Trip,Look for dolphins as you cruise the Indian Oce...,5555
3,64fc46763d690a3e195ee6c6,https://res.cloudinary.com/dwjoypmhg/image/upl...,Lunch with Elephants : Sheldrick Wildlife Trust,Embark on a transformative journey at the Shel...,6170
8,64fc90f13d690a3e195ee8e0,https://res.cloudinary.com/dwjoypmhg/image/upl...,Kazuri Beads Factory Tour,"Step into the world of Kazuri Beads Factory, w...",6170
10,64fca0063d690a3e195ee937,https://res.cloudinary.com/dwjoypmhg/image/upl...,Art Viewing at Nairobi Gallery,Journey into the heart of Kenyan art and cultu...,6170
13,6507ee68313443081a27234a,https://res.cloudinary.com/dwjoypmhg/image/upl...,Nairobi Mamba Village,Dive into an unforgettable reptilian adventure...,6170


In [52]:
from IPython.display import HTML

# Function to create image tags
def display_image(url):
    return f'<img src="{url}" style="max-width:200px;height:auto;">'

# Apply the function to the 'coverImage' column
recommended_experiences_df['coverImage'] = recommended_experiences_df['coverImage'].apply(display_image)

# Display the DataFrame with images
HTML(recommended_experiences_df.to_html(escape=False))


,_id,coverImage,name,description,displayPrice
0,64dfb10e7792cee05d3328d3,,Wasini Day Trip,"Look for dolphins as you cruise the Indian Ocean then snorkel among rich marine life on this fascinating trip, with transfers from select Diani Beach hotels. Learn about East Africa's dark history on a tour of the Slave Caves in Shimoni fishing village, then board a traditional dhow to sail to Wasini Island and snorkel the Kisite-Mpunguti Marine National Park and Reserve. Stay fueled with lunch, snacks, and drinks.",5555
3,64fc46763d690a3e195ee6c6,,Lunch with Elephants : Sheldrick Wildlife Trust,"Embark on a transformative journey at the Sheldrick Wildlife Trust. This sanctuary stands as a beacon of hope for orphaned elephants and rhinos, providing them with expert care and unwavering love. Witness firsthand the incredible dedication of the caretakers as they nurture these majestic creatures back to health. Immerse yourself in the heartwarming stories of resilience and recovery, and learn about the Trust's pioneering conservation efforts. The educational programs offer a deep understanding of wildlife conservation, inspiring a new generation of stewards for our planet. In this sanctuary of compassion, you have the opportunity to adopt an orphaned elephant and become a part of their extraordinary journey to freedom. Your visit to the Sheldrick Wildlife Trust isn't just a visit; it's a chance to be a force of positive change for our endangered wildlife.",6170
8,64fc90f13d690a3e195ee8e0,,Kazuri Beads Factory Tour,"Step into the world of Kazuri Beads Factory, where creativity meets empowerment. This enchanting workshop is a testament to the skill and artistry of Kenyan women, who craft exquisite ceramic beads and pottery. Witness the intricate process as these talented artisans shape, paint, and glaze each bead by hand. The result? Unique, one-of-a-kind creations that reflect the vibrant culture of Kenya. By supporting Kazuri, you're not just acquiring beautiful jewelry and pottery; you're empowering local women and contributing to sustainable, fair-trade practices. Each purchase directly impacts the lives of these artisans, providing them with fair wages, healthcare, and education. Whether you're seeking a meaningful souvenir or simply appreciating the artistry, a visit to Kazuri Beads Factory is an opportunity to be part of a positive change. Come, experience the beauty of Kenyan craftsmanship and be a part of a story of empowerment and artistry.",6170
10,64fca0063d690a3e195ee937,,Art Viewing at Nairobi Gallery,"Journey into the heart of Kenyan art and culture at the Nairobi Gallery. Housed in a historic building, this enchanting space showcases a diverse collection of contemporary and traditional African art. From vibrant paintings to intricately carved sculptures, every piece tells a story of creativity and heritage. The gallery also features a captivating exhibition on the life and works of renowned Kenyan artist S.M. Ngurangani. It's a rare opportunity to delve into the legacy of a local artistic icon. For history enthusiasts, the gallery is situated in the old PC's office, a colonial-era building with its own rich story. The architecture itself is a window into Nairobi's past. Whether you're an art lover, a history buff, or simply seeking inspiration, the Nairobi Gallery offers a dynamic and enriching experience. Come, be captivated by the artistic tapestry that defines Kenya's cultural landscape. Don't miss this chance to immerse yourself in the vibrant world of Kenyan art at the Nairobi Gallery!",6170
13,6507ee68313443081a27234a,,Nairobi Mamba Village,"Dive into an unforgettable reptilian adventure at Nairobi Mamba Village. This extraordinary sanctuary is a haven for crocodile enthusiasts and nature lovers alike. Witness the sheer power and grace of these ancient creatures during thrilling feeding sessions. But Nairobi Mamba Village offers more than just crocodiles. Its lush gardens and tranquil ponds create a serene atmosphere, per